In [1]:
!pip install -r requirement.txt

In [2]:
import numpy as np
import geatpy as ea
import pandas as pd
import random
from datetime import datetime
import parser
print(ea.__version__)

debug = 1 # debug mode

2.7.0


### READ DATA
TODO: read data from GP

In [3]:
rawData = pd.read_excel(io='./FCI ANZ (2022-07-08) (NBV as at 30 Jun 2022)_v2.xlsx', sheet_name='Raw (portfolio)', engine='openpyxl')

In [4]:
# randomly choose 1000 samples
n = 10000
rawData = rawData.iloc[:30000, :61]

In [5]:
data = rawData.sample(n)
print(data.shape)
data.sample(3)

(10000, 61)


,Business Unit (Contract),Business Unit Am,Unit Id Fz,Cost,Po Num Fz,Mfr Dt Fz,Asset Id,Pool Fz,Product,Birth Unit Id Fz,...,Correct CEU,NP year,NBV check,age at drawdown X CEU,RML at drawdown X CEU,category,string,filter 2017 contracts,contract units,filter no. of units
2117,FZOPS,DF3RM,FSCU7160308,3832.0,CIMC-FL-1801,12/18/2017,1069481.0,DFC3,D4H,FSCU7160308,...,1.7,2018 NP,Reasonable,8.977164,11.422836,from ANZ portfolio,LF-HAPAG-09D4H,NaN,2516.0,1.0
2931,FZOPS,DF3RM,FDCU0062928,3682.0,DFIC-FL-1801,2/28/2018,1122976.0,DFC3,D4H,FDCU0062928,...,1.7,2018 NP,Reasonable,8.637164,15.886849,from ANZ portfolio,LT-MSC-35BD4H,NaN,5621.0,1.0
15371,FZOPS,DF3RM,FFAU4032647,6000.0,CSIH-FL-2115,4/8/2021,1706161.0,DFC3,D4H,FFAU4032647,...,1.7,2021 NP,Reasonable,3.367164,10.637808,from ANZ portfolio,LT-MSC-48D4H,NaN,6997.0,1.0


### DATA PREPARATION

In [6]:
# TODO: add more columns later
input_data = \
    data.loc[:, ['Cost', 'Mfr Dt Fz', 'Product', 'Contract Cust Id', 'Contract Dt', 'Contract Expiration Dt', 'Nbv', 'Billing Status Fz']]
input_data.head(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
22812,2267.0,3/22/2018,D20,ONE,2018-02-01,2026-10-14,1896.088,ON
14453,6000.0,4/8/2021,D4H,MSC,2021-04-01,2029-06-30,5612.350,ON
21410,3849.0,4/6/2018,D4H,ONE,2018-02-01,2026-10-14,3123.459,ON


#### Find standard equipment

Assign to `Product`

In [7]:
# select standard equipment
standardEquipment = ['D4H', 'D20', 'D40']
def SelectStandardEquip(Product):
    if Product in standardEquipment:
        Product = 1
    else:
        Product = 0
    return Product

input_data['Product'] = input_data.apply(lambda x: SelectStandardEquip(x['Product']), axis=1)
input_data.head(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
22812,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1896.088,ON
14453,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.350,ON
21410,3849.0,4/6/2018,1,ONE,2018-02-01,2026-10-14,3123.459,ON


#### Find Top 1 and Top 3 Lessees

Assign new column `Top1Lessee` and `Top3Lessees`

In [8]:
# select the Top1 lessee
input_data.groupby('Contract Cust Id').count().sort_values('Cost', ascending=False)

,Cost,Mfr Dt Fz,Product,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz
Contract Cust Id,,,,,,,
MSC,4086,4086,4086,4086,4086,4086,4086
ONE,3676,3676,3676,3676,3676,3676,3676
HAPAG,2028,2028,2028,2028,2028,2028,2028
TCLC,83,83,83,83,83,83,83
COSMR,50,50,50,50,50,50,50
CMA,41,41,41,41,41,41,41
PANOCEAN,36,36,36,36,36,36,36


In [9]:
# top1 and top3 lessees 
Top1Lessee = 'MSC'
Top3Lessees = ['MSC', 'ONE', 'HAPAG']

In [10]:
def SelectTop1(custID):
    if custID == Top1Lessee:
        custID = 1
    else:
        custID = 0
    return custID

def SelectTop3(custID):
    if custID in Top3Lessees:
        custID = 1
    else:
        custID = 0
    return custID

input_data['Top1Lessee'] = input_data.apply(lambda x: SelectTop1(x['Contract Cust Id']), axis=1)
input_data['Top3Lessees'] = input_data.apply(lambda x: SelectTop3(x['Contract Cust Id']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees
161,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.836,ON,0,1
11936,4420.0,3/3/2021,1,ONE,2020-11-16,2029-05-15,4117.430,ON,0,1
1355,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.838,ON,0,1


#### Calcualte manufacture age
Assign new column `MfrAge`

In [11]:
# TODO: consider timezone, consider time_now in the future
time_now = datetime.now()
# calculate manufacture age
date_format = '%m/%d/%Y'
# return in years, round to 2 decimal
def CalculateManufacutreAge(date):
    diff = time_now.date() - datetime.strptime(date, date_format).date()
    return round(diff.days / 365, 2)

input_data['MfrAge'] = input_data.apply(lambda x: CalculateManufacutreAge(x['Mfr Dt Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge
18029,6000.0,4/12/2021,1,MSC,2021-04-01,2029-06-30,5612.35,ON,1,1,1.46
15209,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.35,ON,1,1,1.47
11542,4420.0,3/3/2021,1,ONE,2020-11-16,2029-05-15,4117.43,ON,0,1,1.57


#### Calculate unexpired year 
Assign new column `ExpYear`

In [12]:
# TODO: consider timezone, consider time_now in the future
# TODO: 'Contract Expiration Dt' is in Timestamp, the data type may change
time_now = datetime.now()
# calculate manufacture age
date_format = '%Y-%m-%d'
# return in years, round to 2 decimal
def CalculateExpYear(date):
    diff = pd.to_datetime(date,format="%Y%m%d").date() - time_now.date()
    return round(diff.days / 365, 2)

input_data['ExpYear'] = input_data.apply(lambda x: CalculateExpYear(x['Contract Expiration Dt']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
7738,3702.0,6/11/2018,1,MSC,2018-03-01,2032-07-31,2983.214,ON,1,1,4.30,9.85
29273,2182.0,1/2/2018,1,HAPAG,2017-11-01,2030-01-31,1784.486,ON,0,1,4.74,7.35
19515,3849.0,4/6/2018,1,ONE,2018-02-01,2026-10-14,3123.450,ON,0,1,4.48,4.05


Select billing status

In [13]:
def SelectBilling(status):
    if status == 'ON':
        status = 1
    else:
        status = 0
    return status

input_data['Billing Status Fz'] = input_data.apply(lambda x: SelectBilling(x['Billing Status Fz']), axis=1)
input_data.sample(3)

,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
29592,2182.0,1/2/2018,1,HAPAG,2017-11-01,2030-01-31,1784.490,1,0,1,4.74,7.35
762,3832.0,12/18/2017,1,HAPAG,2017-11-01,2030-01-31,2981.834,1,0,1,4.78,7.35
29254,2182.0,1/2/2018,1,HAPAG,2017-11-01,2030-01-31,1784.486,1,0,1,4.74,7.35


Save data

In [14]:
input_data.to_csv('prepared_data.csv')

In [15]:
prepared_data = pd.read_csv('./prepared_data.csv')
prepared_data.head(3)

,Unnamed: 0,Cost,Mfr Dt Fz,Product,Contract Cust Id,Contract Dt,Contract Expiration Dt,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
0,22812,2267.0,3/22/2018,1,ONE,2018-02-01,2026-10-14,1896.088,1,0,1,4.52,4.05
1,14453,6000.0,4/8/2021,1,MSC,2021-04-01,2029-06-30,5612.350,1,1,1,1.47,6.76
2,21410,3849.0,4/6/2018,1,ONE,2018-02-01,2026-10-14,3123.459,1,0,1,4.48,4.05


### Model Part

#### Input

Convert pandas to numpy array

TODO: set input, may consider alt way.

In [16]:
prepared_data.describe()

,Unnamed: 0,Cost,Product,Nbv,Billing Status Fz,Top1Lessee,Top3Lessees,MfrAge,ExpYear
count,10000.000000,10000.000000,10000.0,10000.000000,10000.0,10000.0000,10000.000000,10000.000000,10000.000000
mean,14997.166600,3973.273543,1.0,3429.066835,1.0,0.4086,0.979000,3.530802,6.687322
std,8757.031075,1230.264743,0.0,1280.471184,0.0,0.4916,0.143391,1.489482,2.023800
min,2.000000,2182.000000,1.0,1784.484000,1.0,0.0000,0.000000,1.350000,1.010000
25%,7359.750000,3682.000000,1.0,2869.540000,1.0,0.0000,1.000000,1.510000,4.050000
50%,14988.500000,3832.000000,1.0,2981.834000,1.0,0.0000,1.000000,4.520000,6.760000
75%,22631.250000,4420.000000,1.0,4136.340000,1.0,1.0000,1.000000,4.730000,7.350000
max,29997.000000,6007.000000,1.0,5618.800000,1.0,1.0000,1.000000,4.790000,9.850000


In [17]:
cost = prepared_data['Cost'].to_numpy().reshape(n, 1)
nbv = prepared_data['Nbv'].to_numpy().reshape(n, 1)
product = prepared_data['Product'].to_numpy()
mfrAge = prepared_data['MfrAge'].to_numpy().reshape(n, 1)
lesseStatus = prepared_data['Billing Status Fz'].to_numpy()
expYear = prepared_data['ExpYear'].to_numpy().reshape(n, 1)
top1 = prepared_data['Top1Lessee'].to_numpy()
top3 = prepared_data['Top3Lessees'].to_numpy()

if debug:
    print('nbv shape: ', nbv.shape)
    print('product shape: ', product.shape)
    print('mfrAge shape: ', mfrAge.shape)
    print('lesseStatus shape: ', lesseStatus.shape)
    print('expYear shape: ', expYear.shape)


nbv shape:  (10000, 1)
product shape:  (10000,)
mfrAge shape:  (10000, 1)
lesseStatus shape:  (10000,)
expYear shape:  (10000, 1)


#### Constraints

`x`: np.array, size=num_of_container. `x[i]=1` stands for $i^{th}$ container is selected; `x[i]=0` stands for $i^{th}$ container is not selected.

1. NBV of standard equip >= 80%: `sum(x*Product*Nbv) / sum(x*Nbv) >= 0.8`

2. Avg age of equip <= 3.75: TODO

3. Avg age of Manufacturing < 7: `sum(x*MfrAge) / sum(x) <= 7`

4. Equipment port subject to lease >= 90%: TODO

5. Avg unexpired term under leases >= 5: `sum(x*Billing Status Fz*ExpYear) / sum(x*Billing Status Fz) >= 5`

6. Top 1 NBV <= 50%: `sum(x*Top1Lessee*Nbv) / sum(x*Nbv) <= 0.5`

7. Top 3 NBV <= 70%: `sum(x*Top3Lessee*Nbv) / sum(x*Nbv) <= 0.7`

#### Objective

min Cost: `min sum(x*Cost)`

#### Define Problem
Define objective function and constraint functions

In [25]:
def DEMO():
    class MyProblem(ea.Problem):
        def __init__(self):
            name ='MyProblem'
            M = 1
            maxormins = [1] # -1: max, 1: min
            Dim = n # num of variables
            varTypes = [1] * Dim # integers: 0 or 1
            lb = [0] * Dim # low bound
            ub = [1] * Dim # up bound
            lbin = [1] * Dim # low bound include
            ubin = [1] * Dim # up bound include
            ea.Problem.__init__(self, name, M, maxormins, Dim, varTypes, lb,ub, lbin, ubin)
            
        def aimFunc(self, pop):
            Vars = pop.Phen
            
            # two optimization methods are showed below. now we only care the first one.
            pop.ObjV = Vars @ cost

            pop.CV = np.hstack([
                -((Vars * product @ nbv) / (Vars @ nbv) - 0.8),
                (Vars @ mfrAge) / Vars.sum(axis=1, keepdims=True) - 7,
                -((Vars * lesseStatus @ expYear) / (Vars * lesseStatus).sum(axis=1, keepdims=True) - 5),
                (Vars * top1 @ nbv) / (Vars @ nbv) - 0.5,
                (Vars * top3 @ nbv) / (Vars @ nbv) - 0.7
            ])




    problem = MyProblem()
    Encoding ='RI'
    NIND = 10000 # init number of population instance
    Field = ea.crtfld(Encoding, problem.varTypes, problem.ranges,problem.borders)
    population = ea.Population(Encoding, Field, NIND)

    # set parameters
    myAlgorithm = ea.soea_DE_best_1_L_templet(problem, population)
    myAlgorithm.MAXGEN = 10000
    myAlgorithm.mutOper.F = 0.5
    myAlgorithm.recOper.XOVR = 0.7
    myAlgorithm.logTras = 1
    myAlgorithm.trappedValue = 1e-6
    myAlgorithm.maxTrappedCount = 1000

    # TODO: random seed has a significant influence on final result!
    # outputMsg=True to show the result
    res = ea.optimize(myAlgorithm, seed=2049, verbose=False, drawing=1, outputMsg=True, drawLog=False, saveFlag=False, dirName='result')
    if debug:
        print(res)
    return res

In [ ]:
# run and debug
selectedPackage = DEMO()

#### Output

In [ ]:
print('Time Consumption: ', np.round(selectedPackage['executeTime'], decimals=2), 's')
print(selectedPackage['stopMsg'])

if selectedPackage['success']:
    print('Algorithm Succeeded!')
    print('===================================================================================================')
    print('Optimal Target Value: ', np.round(selectedPackage['ObjV'].item(), decimals=2))
    print('Constraint Violation: ', np.round(selectedPackage['CV'][0], decimals=2))
    if debug:
        # TODO: doing
        print('===================================================================================================')
        Vars = selectedPackage['Vars'][0]
        print('NBV of standard equip:', np.round((Vars * product @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Avg age of Manufacturing:', np.round((Vars @ mfrAge) / Vars.sum(), decimals=2)[0])
        print('Avg unexpired term under leases:', np.round((Vars * lesseStatus @ expYear) / (Vars * lesseStatus).sum(), decimals=2)[0])
        print('Top 1 NBV:', np.round((Vars * top1 @ nbv) / (Vars @ nbv), decimals=2)[0])
        print('Top 3 NBV:', np.round((Vars * top3 @ nbv) / (Vars @ nbv), decimals=2)[0])
        
else:
    print('No Feasible Results...')

In [ ]:
# print final result
print('Selected Package:')
print('===================================================================================================')
print(len(selectedPackage['Vars'].nonzero()[1]), 'containers selected.')
print(selectedPackage['Vars'].nonzero()[1]) # index begins from 0
print('===================================================================================================')
print(selectedPackage['Vars'][0].tolist())